In [ ]:
from models.modules.unet import Unet
import torch
import torchinfo

net = Unet(
    in_channels=2,
    out_channels=2,
    hidden_channels=[8,10],
    n_enc_cab=[2,2,2],
    n_dec_cab=[2,2,2],
    n_skip_cab=[1,1,1],
    n_cab_bottle=3
)

x = torch.randn(1, 5,1,2,100,100)

print(x)

x = net(x)

print(x)

tensor([[[[[[ 1.4015e+00,  2.2392e-01, -7.1368e-01,  ...,  5.4553e-01,
              8.0008e-01,  2.2032e+00],
            [ 6.2017e-01,  1.3222e+00, -1.4368e+00,  ..., -8.0680e-01,
             -1.1942e+00, -5.3413e-01],
            [ 2.6717e-01, -5.1639e-01,  6.3839e-02,  ...,  1.6853e-01,
              4.3607e-01, -4.7162e-01],
            ...,
            [ 9.9862e-01,  1.9589e-01, -1.4683e-01,  ..., -8.3430e-01,
              1.1458e+00,  4.7351e-01],
            [ 1.1168e+00,  1.2972e+00,  8.6369e-01,  ..., -1.3262e+00,
             -3.4712e-01,  4.0201e-01],
            [ 8.3891e-01,  5.9497e-01, -6.9541e-01,  ..., -2.3239e+00,
             -1.0345e+00,  1.0839e+00]],

           [[-7.4218e-01,  2.1893e-01,  1.9834e+00,  ...,  3.1377e-01,
              4.8416e-01,  1.2442e+00],
            [-1.3487e-01,  2.5122e-01, -2.6674e-01,  ..., -3.3199e-01,
              4.0937e-01, -7.4979e-01],
            [-2.3918e+00,  3.2513e-02,  5.1589e-01,  ...,  2.9973e-02,
             -8.1205e-

In [6]:
from models.modules.context import ExtraContext, register_extra_loss, register_extra_hook, get_extra_context

from torch import nn
import torch

# In sub module:
class SubModel(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        register_extra_loss(self, x.mean(), "localloss")

        def dosomething_hook():
            return x.mean()
        register_extra_hook(self, dosomething_hook, "This is a submodel hook")
        if ctx := get_extra_context(self):
            ctx['local_var'] = 'This is a local variable'
        return x

# In top module:
class TopModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.subm = SubModel()
    
    def forward(self, x):
        x = self.subm(x)
        return x

    
# In training step:
batch = torch.randn(10, 3, 32, 32, requires_grad=True)  # Example input batch
target = batch

lossfn = lambda x, y: x.mean()

model = TopModel()
with ExtraContext(model) as ctx:
    pred = model(batch)

    # ... training step

    losses = ctx.losses
    hooks = ctx.hooks
    # losses, hooks, objects are lists of tuples:
    # (prefix, loss) for every loss
    # (prefix, hook) for every hook

    loss = lossfn(pred, target)
    for prefix, localoss in losses:
        print(f"Loss {prefix}: {loss.item()}")
        loss += localoss
    loss.backward()


Loss localloss: -0.001828571199439466


In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU()
        )
        self.layer2 = nn.Linear(16, 10)

class TopModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.submodel = MyModel()
        self.submodel2 = self.submodel
        self.submodel3 = self.submodel

model = TopModel()

# 遍历模型的模块及其前缀
for prefix, module in model.named_modules():
    print(f"Prefix: {prefix}, Module: {module}")

In [ ]:
import torch

x = torch.randn([1231])

x.extraloss = torch.randn([1231, 2])

print(x.extraloss)

y = x + 1

print(y.extraloss)

In [ ]:
class CustomModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.param = nn.Parameter(torch.tensor(1.0))
        self.extra_data = {"custom_key": 123}  # 自定义状态

    def get_extra_state(self):
        # 返回额外状态
        return self.extra_data

    def set_extra_state(self, state):
        # 恢复额外状态
        self.extra_data = state

model = CustomModule()
# 保存模型
torch.save(model.state_dict(), "model.pth")

# 保存额外状态
torch.save(model.get_extra_state(), "extra_state.pth")

# 加载模型
extra_state = torch.load("extra_state.pth")
model.set_extra_state(extra_state)


In [ ]:
from torch import nn
import torch

class SubModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.loss = torch.tensor(0.0)

    def state_dict(self, destination=None, prefix='', keep_vars=False):
        state = super().state_dict(destination = destination, prefix = prefix, keep_vars = keep_vars)
        state[prefix + 'loss'] = self.loss
        return state

    def forward(self, x):
        loss = x.mean()

        def get_loss():
            return loss
        
        self.get_loss = get_loss

        return x

class TopModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.subModule = SubModule()

    def forward(self, x):
        x = self.subModule(x)
        return x

tb = TopModule()

x = torch.randn(2,2, requires_grad=True)
x = tb(x)

for m in tb.modules():
    if hasattr(m, "get_loss"):
        print(m.get_loss())

In [ ]:
import torch
import torch.nn as nn

class SubModule(nn.Module):
    def __init__(self):
        super(SubModule, self).__init__()
        self.sub_param = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
        
class ParentModule(nn.Module):
    def __init__(self):
        super(ParentModule, self).__init__()
        self.parent_param = nn.Parameter(torch.tensor([4.0, 5.0, 6.0]))
        self.sub_module = SubModule()  # 子模块
        
parent_model = ParentModule()

# 访问父模块的 state_dict
state_dict = parent_model.state_dict()
print(state_dict)

In [ ]:
from models.vqpromptunet import VQPromptUnet

m = VQPromptUnet(2,2,4,[2,4,6],[2,2,2],[2,2,2],[2,2,2],[2,2,2],[2,2,2],[2,2,2],3)

In [ ]:
from glob import glob
import os

path = "/home/hulabdl/CMRxRecon2025/R1_mat_folder/"

filelist = glob(os.path.join(path, "*.mat"))
filelist = [file for file in filelist if 'mask' not in file.lower()]

scorepath = "/home/hulabdl/PromptMR-plus/scorelist.txt"

scores = []

with open(scorepath, 'r') as f:
    for line in f:
        scores.append(line.strip().split())

res = []

for cond in scores:
    for file in filelist:
        flag = True
        for keyword in cond:
            if keyword.lower() not in file.lower():
                flag = False

        if flag:
            res.append((cond, file))


for line in res:
    print(line)

print(len(res))


In [ ]:
import h5py

for cond, file in res:
    with h5py.File(file, 'r') as f:
        data = f['kus']
        shape = data.shape
        if len(shape) == 4:
            shape = [1] + list(shape)
        print(file, "\t".join(map(str, shape)), sep="\t")


In [ ]:
import os

def get_mat_files(directory):
    mat_files = []
    # 遍历目录及其子目录
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".mat") and "kus" in file:  # 检查文件扩展名
                mat_files.append(os.path.join(root, file))  # 获取完整路径
    return mat_files

# 示例用法
directory = "/home/hulabdl/PromptMR-plus/valres1"
mat_files = get_mat_files(directory)
print(f"Found .mat files:{len(mat_files)}")

In [ ]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/T1rho/TrainingSet/FullSample/Center001/UIH_30T_umr780/P002/T1rho.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data.keys())
print(data['kspace'].shape)

# from matplotlib import pyplot as plt
# import numpy as np

# print(np.sum(data['mask'][0], axis=0)[0]-21) 
# plt.imshow(data['mask'][0])
# plt.show()

In [ ]:
import torch

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

for mat in mat_files:
    data = loadmat(mat)
    recon = torch.as_tensor(data['img4ranking'])
    is_nan = not torch.isfinite(recon).any()
    if is_nan:
        print(mat)

In [ ]:
# path='/home/hulabdl/PromptMR-plus/blackblood_kus_ktRadial16.mat'
path = "/home/hulabdl/PromptMR-plus/blackblood_kus_ktRadial8.mat"
# path = "/nas-data/datasets/CMRxRecon2025/TaskR1/MultiCoil/BlackBlood/ValidationSet/UnderSample_TaskR1/Center006/Siemens_30T_Prisma/P018/blackblood_kus_ktRadial16.mat"

import h5py
from scipy.io import loadmat as loadmat_scipy
from matplotlib import pyplot as plt




data = loadmat(path)
print(data.keys())
print(data['img4ranking'].shape)

import torch

tensor = torch.as_tensor(data['img4ranking'])

print(tensor)

plt.figure()
plt.imshow(data['img4ranking'][1], cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
path='/home/hulabdl/CMRxRecon2025/ChallengeData/MultiCoil/T1w/TrainingSet/Mask_TaskAll/Center003/UIH_30T_umr880/P004/T1w_mask_ktRadial16.mat'

import h5py
from scipy.io import loadmat as loadmat_scipy

def loadmat_group(group):
    """
    Load a group in Matlab v7.3 format .mat file using h5py.
    """
    data = {}
    for k, v in group.items():
        if isinstance(v, h5py.Dataset):
            data[k] = v[()]
        elif isinstance(v, h5py.Group):
            data[k] = loadmat_group(v)
    return data

def loadmat(filename):
    """
    Load Matlab v7.3 format .mat file using h5py.
    """
    try:
        with h5py.File(filename, 'r') as f:
            data = {}
            for k, v in f.items():
                if isinstance(v, h5py.Dataset):
                    data[k] = v[()]
                elif isinstance(v, h5py.Group):
                    data[k] = loadmat_group(v)
    except OSError as e:
        f = loadmat_scipy(filename)
        data = {key: value for key, value in f.items() if not key.startswith('__')}
    return data

data = loadmat(path)
print(data['mask'].shape)